# The PolInSAR Course - June 3rd, 2024
# SAR Polarimetry (PolSAR) 
# Part 2: Eigenvalues of the Polarimetric Coherency Matrix and the Entropy/Anisotropy/Alpha decomposition

* Acquisition: Nkok (Gabon), DLR's F-SAR, L-band

* Path to images: /projects/data/polsar/

* SLC (single-look complex) images:
    * HH: slc_16afrisr0107_Lhh_tcal_test.rat
    * HV: slc_16afrisr0107_Lhv_tcal_test.rat
    * VH: slc_16afrisr0107_Lvh_tcal_test.rat
    * VV: slc_16afrisr0107_Lvv_tcal_test.rat

Tips:
- use a function that performs the multilook (correlation) operation on a moving window with (looksa x looksr) pixels in range - azimuth
- focus on a azimuth - range block within pixels [5000, 15000] and [0, 2000], respectively.

In [ ]:
# --- Download exercise data & import reader function
from pysarpro import io, data
from pysarpro.io import rrat

data.download_all(directory="/projects", pattern=r'^data/polsar')

# --- Import useful libaries, functions, and modules
import sys
sys.path.append('/projects/src/')
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import uniform_filter
%matplotlib widget

**Auxiliary functions**

`HSV_colormap_to_rgb`: Generates and HSV composite representation based on a given colormap.

In [ ]:
def HSV_colormap_to_rgb(colormap, h, s, v):
    """
    Makes an HSV-like RGB representation based on the given colormap instead
    of 'hsv' colormap.
    
    See https://en.wikipedia.org/wiki/HSL_and_HSV

    Parameters
    ----------
    colormap : function
        Colormap function. Takes the values in 'h' array and returns an RGBA
        value for each point. The ones in matplotlib.cm should be compatible
    h : ndarray
        Hue values. Usually between 0 and 1.0.
    s : ndarray
        Saturation values. Between 0 and 1.0.
    v : ndarray
        Value values. Between 0 and 1.0.

    Returns
    -------
    rgb: ndarray
        An array with the same shape as input + (3,) representing the RGB.
    """
    # Generate color between given colormap (colormap(h)) and white (ones)
    # according to the given saturation
    tmp = (1-s)[..., np.newaxis]*np.ones(3) + s[..., np.newaxis] * colormap(h)[...,:3]
    # Scale it by value
    return v[..., np.newaxis] * tmp

`calculate_covariance`: Calculates the covariance between two images while performing a multi-looking operation.

In [ ]:
def calculate_covariance(im1, im2, looksa, looksr):
    
     # ... apply definition
    corr = uniform_filter( np.real(im1*np.conj(im2)), [looksa, looksr] ) + \
        1j*uniform_filter( np.imag(im1*np.conj(im2)), [looksa, looksr] )
    
    # ... and back to main
    return corr

`calculate_eigenvalues_3`: Computes the eigenvalues of a 3x3 matrix analytically. 

In [ ]:
def calculate_eigenvalues_3(T11, T12, T13, T22, T23, T33):

    # Calculate and order (from max to min) the eigenvalues of a 3x3 hermitian matrix in closed-form.
    # Inputs can be 2D az - rg (rows - columns).

    # get dimensions
    dims = T11.shape

    # calculate auxiliary quantities
    A = T11*T22 + T11*T33 + T22*T33 - T12*np.conj(T12) - T13*np.conj(T13) - T23*np.conj(T23)
    B = T11**2 - T11*T22 + T22**2 -T11*T33 -T22*T33 + T33**2 + 3*T12*np.conj(T12) + 3*T13*np.conj(T13) + 3*T23*np.conj(T23)

    DET = T11*T22*T33 - T33*T12*np.conj(T12) - T22*T13*np.conj(T13) - T11*T23*np.conj(T23) + T12*np.conj(T13)*T23 + np.conj(T12)*T13*np.conj(T23)  
    TR = T11 + T22 + T33 
    Z = 27*DET-9*A*TR + 2*TR**3 + np.sqrt((27*DET-9*A*TR + 2*TR**3)**2-4*B**3)
    
    del DET
    
    # ... and here they are:
    LA = ( 1/3.*TR + 2**(1/3.)*B/(3*Z**(1/3.)) + Z**(1/3.)/(3*2**(1/3.)) )
    LB = ( 1/3.*TR - (1+1j*np.sqrt(3))*B/(3*2**(2/3.)*Z**(1/3.)) - (1-1j*np.sqrt(3))*Z**(1/3.)/(6*2**(1/3.)) )
    LC = ( 1/3.*TR - (1-1j*np.sqrt(3))*B/(3*2**(2/3.)*Z**(1/3.)) - (1+1j*np.sqrt(3))*Z**(1/3.)/(6*2**(1/3.)) )
    
    # now order them:
    dumm = np.zeros((dims[0], dims[1], 3), 'float32')
    dumm [:, :, 0] = np.real(LA)
    dumm [:, :, 1] = np.real(LB)
    dumm [:, :, 2] = np.real(LC)
    
    del LA, LB, LC  
    
    L1 = np.max(dumm, axis = 2)
    L3 = np.min(dumm, axis = 2)
    L2 = np.sum(dumm, axis = 2) - L1 - L3
    
    del dumm
    
    return L1, L2, L3
    

`calculate_eigenvectors_3`: Computes the eigenvectors of a 3x3 matrix analytically. 

In [ ]:
def calculate_eigenvectors_3(T11, T12, T13, T22, T23, T33, L1, L2, L3) :

    # Calculate the eigenvectors corresponding to the eigenvalues (L1, L2, L3)
    # of a 3x3 matrix 
    # Inputs can be 2D az - rg (rows - columns).

    # get dimensions
    dims = T11.shape    
    
    # first eigenvector - corresponds to the maximum eigenvalue L1
    U1 = np.ones((dims[0], dims[1], 3), 'complex64')
    U1[:, :, 0] = (L1 -T33)/np.conj(T13) + (((L1-T33)*np.conj(T12) + np.conj(T13)*T23)*np.conj(T23))/ \
                    (((T22-L1)*np.conj(T13) - np.conj(T12)*np.conj(T23))*np.conj(T13))
    U1[:, :, 1] = -((L1-T33)*np.conj(T12)+np.conj(T13)*T23) / ((T22-L1)*np.conj(T13) - np.conj(T12)*np.conj(T23))
    
    # second eigenvector - corresponds to the eigenvalue L2
    U2 = np.ones((dims[0], dims[1], 3), 'complex64')
    U2[:, :, 0] = (L2 -T33)/np.conj(T13) + (((L2-T33)*np.conj(T12) + np.conj(T13)*T23)*np.conj(T23))/ \
                    (((T22-L2)*np.conj(T13) - np.conj(T12)*np.conj(T23))*np.conj(T13))
    U2[:, :, 1] = -((L2-T33)*np.conj(T12)+np.conj(T13)*T23) / ((T22-L2)*np.conj(T13) - np.conj(T12)*np.conj(T23))
    
    # third eigenvector - corresponds to the minimum eigenvalue L3
    U3 = np.ones((dims[0], dims[1], 3), 'complex64')
    U3[:, :, 0] = (L3 -T33)/np.conj(T13) + (((L3-T33)*np.conj(T12) + np.conj(T13)*T23)*np.conj(T23))/ \
                    (((T22-L3)*np.conj(T13) - np.conj(T12)*np.conj(T23))*np.conj(T13))
    U3[:, :, 1] = -((L3-T33)*np.conj(T12)+np.conj(T13)*T23) / ((T22-L3)*np.conj(T13) - np.conj(T12)*np.conj(T23))   
    
    # normalize to get orthonormal eigenvectors
    norm1 = np.sqrt( np.abs(U1[:,:,0])**2 + np.abs(U1[:,:,1])**2 + np.abs(U1[:,:,2])**2)
    norm2 = np.sqrt( np.abs(U2[:,:,0])**2 + np.abs(U2[:,:,1])**2 + np.abs(U2[:,:,2])**2)    
    norm3 = np.sqrt( np.abs(U3[:,:,0])**2 + np.abs(U3[:,:,1])**2 + np.abs(U3[:,:,2])**2)        
    for nn in range(3):
        U1[:,:,nn] = U1[:,:,nn] / norm1
        U2[:,:,nn] = U2[:,:,nn] / norm2
        U3[:,:,nn] = U3[:,:,nn] / norm3
        
    del norm1, norm2, norm3     
    
    return U1, U2, U3


**Input parameters**

In [ ]:
# path to the data
path = '/projects/data/polsar/'
# define the number of looks 
looksa = 7
looksr = 7

**Step 1: Load data**

In [ ]:
slcHH = rrat(path + 'slc_16afrisr0107_Lhh_tcal_test.rat', block=[5000,15000,0,2000])
slcHV = rrat(path + 'slc_16afrisr0107_Lhv_tcal_test.rat', block=[5000,15000,0,2000])
slcVV = rrat(path + 'slc_16afrisr0107_Lvv_tcal_test.rat', block=[5000,15000,0,2000])

In [ ]:
# check shape
slcHH.shape

**Step 2: Calculate the necessary elements of the coherency matrix**

In [ ]:
# -- compute the Pauli components
pauli1 = slcHH + slcVV
pauli2 = slcHH - slcVV
pauli3 = 2*slcHV

In [ ]:
# -- compute the elements of the coherency matrix
T11 = calculate_covariance(pauli1, pauli1, looksa, looksr)
T22 = calculate_covariance(pauli2, pauli2, looksa, looksr)
T33 = calculate_covariance(pauli3, pauli3, looksa, looksr)
T12 = calculate_covariance(pauli1, pauli2, looksa, looksr)
T13 = calculate_covariance(pauli1, pauli3, looksa, looksr)
T23 = calculate_covariance(pauli2, pauli3, looksa, looksr)

In [ ]:
# -- delete unused variables
del slcHH, slcHV, slcVV
del pauli1, pauli2, pauli3

**Step 3: Calculate eigenvalues**

In [ ]:
lambda1, lambda2, lambda3 = calculate_eigenvalues_3(T11, T12, T13, T22, T23, T33)

In [ ]:
# check shape
lambda1.shape

**Step 4: Calculate entropy**

In [ ]:
# -- compute the probabilities associated with each eigenvalue
pr1 = lambda1 / (lambda1 + lambda2 + lambda3)
pr2 = lambda2 / (lambda1 + lambda2 + lambda3)
pr3 = lambda3 / (lambda1 + lambda2 + lambda3)

In [ ]:
# -- compute the entropy
entropy = -(pr1 * np.log10(pr1)/np.log10(3) + pr2 * np.log10(pr2)/np.log10(3) + pr3 * np.log10(pr3)/np.log10(3))

**Step 5: Calculate anisotropy** 

In [ ]:
# -- compute the anisotropy (related to the minimum and intermediate eigenvalues)
# A = 0 when lambda2 = lambda3
# A = 1 when lambda2 >> lambda3 
anisotropy =  (lambda2 - lambda3) / (lambda2 + lambda3)

**Step 6: Calculate eigenvectors**

In [ ]:
# -- compute the eigenvectors
U1, U2, U3 = calculate_eigenvectors_3(T11, T12, T13, T22, T23, T33, lambda1, lambda2, lambda3)

In [ ]:
# check shape
U1.shape

In [ ]:
# -- delete unused variables
del T12, T13, T23

**Step 7: Calculate mean alpha angle**

In [ ]:
# -- extract the alpha angles
alpha1 = np.arccos(np.abs(U1[:,:,0]))  # [rad]
alpha2 = np.arccos(np.abs(U2[:,:,0]))
alpha3 = np.arccos(np.abs(U3[:,:,0]))

In [ ]:
# -- delete unused variables
del U1, U2, U3

In [ ]:
# -- compute the mean alpha angle
alpha_mean = pr1*alpha1 + pr2*alpha2 + pr3*alpha3
# -- transfer to degrees
alpha_mean = np.degrees(alpha_mean)

**Step 8: Plots!**

In [ ]:
# Calculations for Paulis RGB:
# -- define the 3D array for the Pauli representation
dimaz = T11.shape[0]
dimrg = T11.shape[1]
rgb_pauli = np.zeros((dimrg, dimaz,3), 'float32')

# -- fill the array, clipping the values between 0 and 2.5xmean(amplitude)
rgb_pauli[:,:,0] =np.clip(np.transpose(np.sqrt(abs(T22))), 0, 2.5*np.mean(np.sqrt(abs(T22)))) # R: HH-VV
rgb_pauli[:,:,1] =np.clip(np.transpose(np.sqrt(abs(T33))), 0, 2.5*np.mean(np.sqrt(abs(T33)))) # G: HV
rgb_pauli[:,:,2] =np.clip(np.transpose(np.sqrt(abs(T11))), 0, 2.5*np.mean(np.sqrt(abs(T11)))) # B: HH+VV

# -- normalisation: values between 0 and 1
rgb_pauli[:,:,0] = rgb_pauli[:,:,0] / np.max(rgb_pauli[:,:,0])
rgb_pauli[:,:,1] = rgb_pauli[:,:,1] / np.max(rgb_pauli[:,:,1])
rgb_pauli[:,:,2] = rgb_pauli[:,:,2] / np.max(rgb_pauli[:,:,2])

In [ ]:
# Plot: Pauli RGB and eigenvalue probabilities
plt.figure(figsize=(15, 6*4))
plt.subplot(4,1,1)
plt.imshow(rgb_pauli, aspect='auto')
plt.colorbar()

plt.subplot(4,1,2)
plt.imshow(np.transpose(pr1), cmap='turbo', vmin=0, vmax=1, aspect='auto')
cb = plt.colorbar()
cb.set_label('pr1')

plt.subplot(4,1,3)
plt.imshow(np.transpose(pr2), cmap='turbo', vmin=0, vmax=1, aspect='auto')
cb = plt.colorbar()
cb.set_label('pr2')

plt.subplot(4,1,4)
plt.imshow(np.transpose(pr3), cmap='turbo', vmin=0, vmax=1, aspect='auto')
cb = plt.colorbar()
cb.set_label('pr3')

In [ ]:
# Plot: H, A, alpha
plt.figure(figsize=(15, 6*3))

plt.subplot(3,1,1)
plt.imshow(np.transpose(entropy), cmap='gray', vmin=0, vmax=1, aspect='auto')
cb = plt.colorbar()
cb.set_label('Entropy H')

plt.subplot(3,1,2)
plt.imshow(np.transpose(anisotropy), cmap='turbo', vmin=0, vmax=1, aspect='auto')
cb = plt.colorbar()
cb.set_label('Anisotropy A')

plt.subplot(3,1,3)
plt.imshow(np.transpose(alpha_mean), cmap='turbo', vmin=0, vmax=90, aspect='auto')
cb = plt.colorbar()
cb.set_label('Mean alpha angle [deg]')

In [ ]:
# HSI Color Representation:


 HSI Color Representation:
- H (hue):  mean alpha angle
- S (saturation): 
     - Case 1: saturation = 1: always full colorscale
     - Case 2:  saturation = 1 - entropy
          - when entropy = 0: then saturation = 1: full colorscale
          - when entropy = 1: then saturation = 0: grayscale
- I (intensity): amplitude of total power

In [ ]:
# Hue: mean alpha angle
# normalize the mean alpha angle: it has to be between 0 and 1 --> divide by 90 degrees
hue = alpha_mean / 90 

# Import the colormap for plotting alpha
colormap = plt.colormaps.get('turbo')

# Intensity: normalize the amplitude
amp = np.sqrt(abs(T11) + abs(T22) + abs(T33))
amp = np.clip(amp, 0, 2.5*np.mean(amp))
amp = amp / np.max(amp)

# Saturation
# Case 1)
sat1 = np.ones_like(amp)
# Case 2)
sat2 = 1 - entropy

In [ ]:
# Generate the HSV colormaps 

# Case 1
rgb_comp1 = HSV_colormap_to_rgb(colormap, hue, sat1, amp)

# Case 2
rgb_comp2 = HSV_colormap_to_rgb(colormap, hue, sat2, amp)

In [ ]:
# Plot: HSI representations

plt.figure(figsize=(12,12))

plt.subplot(2,1,1)
plt.imshow(np.transpose(rgb_comp1, axes=(1,0,2)), aspect = 'auto')

plt.subplot(2,1,2)
plt.imshow(np.transpose(rgb_comp2, axes=(1,0,2)), aspect = 'auto')

plt.tight_layout()